In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [11]:
from google import genai
from google.genai import types

# Define the function declaration
weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco",
            },
        },
        "required": ["location"],
    },
}

def get_current_temperature(location):
    return {"location": location, "temperature": "21°C"}

# Client setup
client = genai.Client()
tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])

# Step 1: User asks a question
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[types.Content(role="user", parts=[types.Part(text="What's the temperature in Bengaluru?")])],
    config=config,
)

tool_call = response.candidates[0].content.parts[0].function_call

if tool_call and tool_call.name == "get_current_temperature":
    result = get_current_temperature(**tool_call.args)
    print(f"Function execution result: {result}")

    # FIX: send function result back in same conversation
    followup_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="What's the temperature in Bengaluru?")]
        ),
        types.Content(
            role="model",
            parts=[types.Part(function_call=tool_call)]
        ),
        types.Content(
            role="function",
            parts=[types.Part(
                function_response=types.FunctionResponse(
                    name=tool_call.name,
                    response=result
                )
            )]
        )
    ],
)
    print("Model’s natural language reply:")
    print(followup_response.candidates[0].content.parts[0].text)

else:
    print("No function call found in the response.")


Function execution result: {'location': 'Bengaluru', 'temperature': '21°C'}
Model’s natural language reply:
The current temperature in Bengaluru is 21°C.


In [13]:
# Function calling with thinking
from google import genai
from google.genai import types

client = genai.Client()

grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

config = types.GenerateContentConfig(
    tools=[grounding_tool],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # ask model to return reasoning
    ),
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List top 10 restaurants in bengaluru which is available in  today(09/30/2025) ",
    config=config,
    
)

print(response.text)


Here are 10 highly-regarded restaurants in Bengaluru that are likely to be open today, September 30, 2025, based on recent reviews and operating information:

1.  **Farmlore** - Known for its "extreme farm-to-table" concept, Farmlore offers a unique dining experience.
2.  **Jamavar at The Leela Palace** - This restaurant is celebrated for its Indian pomp and grandeur, offering Indian cuisine.
3.  **Karavalli** - Specializing in Bangalorean seafood classics, Karavalli provides a distinctive regional experience.
4.  **ZLB23** - Described as India's premier Prohibition-era speakeasy, ZLB23 offers a sophisticated ambiance.
5.  **Copitas** - Located in Bengaluru, Copitas offers finely honed Indian hospitality.
6.  **The Fatty Bao** - This vibrant Asian fusion restaurant in Indiranagar is a must-visit for its street food-inspired dishes with contemporary flavors.
7.  **Rim Naam at The Oberoi** - For a luxurious dining experience, Rim Naam offers a stunning atmosphere with a focus on seafood 

In [20]:
#Inspecting Thought Signatures- mostly it will be displayed when we make function calling, to tell the model what operation performed in before request



import base64
# After receiving a response from a model with thinking enabled
# response = client.models.generate_content(...)

# The signature is attached to the response part containing the function call


for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

        print(base64.b64encode(part.thought_signature).decode("utf-8"))


In [22]:
from google import genai
from google.genai import types

client = genai.Client()

# -------------------------------
# Step 1: Define the function
# -------------------------------
restaurant_rating_function = {
    "name": "get_restaurant_rating",
    "description": "Fetches the average rating for a given restaurant in Bengaluru.",
    "parameters": {
        "type": "object",
        "properties": {
            "restaurant_name": {
                "type": "string",
                "description": "The name of the restaurant.",
            },
        },
        "required": ["restaurant_name"],
    },
}

def get_restaurant_rating(restaurant_name: str):
    # Mock data – in real case, this could query Zomato/Google Maps API
    mock_ratings = {
        "Farmlore": 4.8,
        "Jamavar": 4.6,
        "Karavalli": 4.7,
        "ZLB23": 4.5,
        "Copitas": 4.6,
        "The Fatty Bao": 4.4,
        "Rim Naam": 4.7,
        "Mavalli Tiffin Room": 4.3,
        "Yuki Cocktail Bar & Kitchen": 4.5,
        "Lotus Pavilion": 4.4,
    }
    return {"restaurant": restaurant_name, "rating": mock_ratings.get(restaurant_name, "N/A")}

# -------------------------------
# Step 2: Configure with thinking
# -------------------------------
tools = types.Tool(function_declarations=[restaurant_rating_function])

config = types.GenerateContentConfig(
    tools=[tools],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # Ask the model to return its reasoning
    ),
)

# -------------------------------
# Step 3: Ask the model
# -------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the rating of Farmlore restaurant in Bengaluru?",
    config=config,
)

# -------------------------------
# Step 4: Inspect response
# -------------------------------
for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

    if part.function_call:
        print(f"\nModel requested function: {part.function_call.name}")
        print(f"Arguments: {part.function_call.args}")

        # Execute the function
        result = get_restaurant_rating(**part.function_call.args)
        print(f"Function execution result: {result}")

        # -------------------------------
        # Step 5: Send function result back
        # -------------------------------
        followup_response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                response.candidates[0].content,  # include model’s function call turn
                types.Content(
                    role="function",
                    parts=[types.Part(
                        function_response=types.FunctionResponse(
                            name=part.function_call.name,
                            response=result
                        )
                    )]
                )
            ],
        )

        print("\nFinal model reply:")
        print(followup_response.text)


Found thought signature in part 1: b'\n\xa7\x02\x01\xd1\xed\x8ao\x83\x12\x97\xea+\xfc5,\xe1:\xbd\xd2T-\x04Ux\'\x8cLbs\x96\xff\xcaT\x03\\\x9f\xd7\x93=\nTM\x93\x01OgNS\\\x0e\xa7\xd1\x02c\xea\x1e\xd9\x06\xac\xb0~\x93\xae\xe2^\x85L=\xb0\xcd\x9a\x08x\x91\xd7\xa4F\xa6e\xf2\xe2\xc4\x02\xa4\xf4\xf0\x13P\x8e\xbd\x88TL\xdef\xa9\x8aW&\xbb\x15\xad\x87g\xb1\x10\x87\xacM\xe1\xaed2T\x84\x98.\x16\xa5\xc8\xd5\x0bQ\x91\xa0;\xb3\xa0\x06\xdc\x08\xec\xb7\x98\xde\xf9\xd2\x00#\xb7\xe6\x82\xfb"\xbej\x9dyFY\x97\x94\xddg_\xa6V.f\xf4\xf0p\xce\x8dt\x92!\xb159D\x16\xfb\xac\xa8\xc9\x05\xf8\xb3\xf3~\x11!D-\xad\x04\xf1rY\xa4H\xe2\x804\x1e\x01\xba:RU\xe4\xe7\x9d\xc7G\xd8-(\x0e`\xb6\x9fM\x18\xbd\x86\xe1py.-\xe5\xbc\x8a\xcd;\xdc7\xeb^\r\xbf\x99\x05K\x8b]\x8b\xbb4\xe9\xa8\x98<*8l\x9e\xbfd{\xead\x14\x08\xd3\xa3\x1b\x83v\x92H\xaa\xc9Ox\x1a\xb2_\xd6.\x03\x89\xd0>\t\x08E\xd2C\xa4\xaa\xfeQ\x97\xff\x1e\x8c\x9a\xf5\x97p\xfa,L3'

Model requested function: get_restaurant_rating
Arguments: {'restaurant_name': 'Farmlore'}
Function 

In [3]:
# Automatic function schema declaration

from google import genai
from google.genai import types

def multiply(a: float, b: float):
    """Returns a * b."""
    return a * b

client = genai.Client()
fn_decl = types.FunctionDeclaration.from_callable(callable=multiply, client=client)

# to_json_dict() provides a clean JSON representation.
print(fn_decl.to_json_dict())

{'description': 'Returns a * b.', 'name': 'multiply', 'parameters': {'properties': {'a': {'type': 'NUMBER'}, 'b': {'type': 'NUMBER'}}, 'required': ['a', 'b'], 'type': 'OBJECT'}}
